In [1]:
import os, gc, torch

import numpy as np
import pandas as pd
# from models.load import TrainedModels

from utils.eval import save_iou_results
from utils.engine import xami_evaluate, get_iou_types
from models.load import get_trained_model
from utils.coco_eval import get_eval_params_dict
from data.datasets import  OurRadiologsitsDataset, collate_fn
from our_radiologist.load import get_anns
from utils.coco_utils import get_cocos, get_coco_api_from_dataset
from utils.eval import get_ap_ar, get_num_fps, get_num_fns, get_num_tps
from utils.print import print_title
from utils.init import reproducibility, clean_memory_get_device
from data.load import get_datasets, get_dataloaders
from data.constants import XAMI_MIMIC_PATH, DEFAULT_REFLACX_LABEL_COLS
from utils.constants import full_iou_thrs, iou_thrs_5to95
from data.load  import seed_worker, get_dataloader_g
from tqdm import tqdm


from data.transforms import get_transform
from data.datasets import ReflacxDataset
from torch.utils.data import DataLoader

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [2]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [3]:
from enum import Enum

class TrainedModels(Enum):
    CXR_Clinial_fusion1_fusion2="val_ar_0_5436_ap_0_1911_test_ar_0_5476_ap_0_3168_epoch49_WithClincal_05-23-2022 12-06-22_CXR_Clinical_roi_heads_spatialisation"
    CXR_Clinical_fusion1 = "val_ar_0_5476_ap_0_1984_test_ar_0_6038_ap_0_2757_epoch41_WithClincal_05-30-2022 08-01-54_CXR_Clinical_fusion1"
    CXR_Clinical_fusion2= "val_ar_0_4369_ap_0_2098_test_ar_0_4940_ap_0_2218_epoch58_WithClincal_05-30-2022 13-58-43_CXR_Clinical_fusion2"
    CXR="val_ar_0_5659_ap_0_1741_test_ar_0_5390_ap_0_1961_epoch36_WithoutClincal_05-29-2022 12-29-51_CXR"

In [4]:
naming_map = {
    TrainedModels.CXR: "Mask R-CNN (Baseline)",
    TrainedModels.CXR_Clinical_fusion1: "MSF-Net (3D)",
    TrainedModels.CXR_Clinical_fusion2: "MSF-Net (1D)",
    TrainedModels.CXR_Clinial_fusion1_fusion2: "MDF-Net"
}


In [5]:
normal_iou_thrs = iou_thrs_5to95
all_range_iou_thrs = full_iou_thrs
score_thrs = 0.1

In [6]:
DEFAULT_REFLACX_LABEL_COLS

['Enlarged cardiac silhouette',
 'Atelectasis',
 'Pleural abnormality',
 'Consolidation',
 'Pulmonary edema']

In [7]:
disease = "Pulmonary edema"

In [8]:
num_fps = {}
num_fns = {}
num_tps = {}

In [9]:
for select_model in tqdm([
    TrainedModels.CXR_Clinical_fusion1,
    TrainedModels.CXR_Clinical_fusion2,
    TrainedModels.CXR_Clinial_fusion1_fusion2,
    TrainedModels.CXR
]):
    model, train_info, _, _ = get_trained_model(
            select_model,
            DEFAULT_REFLACX_LABEL_COLS,
            device,
            rpn_nms_thresh=0.3,
            box_detections_per_img=10,
            box_nms_thresh=0.2,
            rpn_score_thresh=0.0,
            box_score_thresh=score_thrs,
        )

    model.eval()


    iou_types = get_iou_types(model, train_info.model_setup)

    model_setup = train_info.model_setup
    dataset_params_dict = {
        "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
        "with_clinical": model_setup.use_clinical,
        "dataset_mode": model_setup.dataset_mode,
        "bbox_to_mask": model_setup.use_mask,
        "normalise_clinical_num": model_setup.normalise_clinical_num,
        "labels_cols": DEFAULT_REFLACX_LABEL_COLS,
    }

    test_dataset = ReflacxDataset(
        **dataset_params_dict, split_str="test", transforms=get_transform(train=False),
    )

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=4,
        shuffle=True,
        collate_fn=collate_fn,
        worker_init_fn=seed_worker,
        generator=get_dataloader_g(0),
    )

    test_coco = get_coco_api_from_dataset(test_dataloader.dataset)

    normal_eval_params_dict = get_eval_params_dict(
            test_dataset, iou_thrs=normal_iou_thrs,
        )


    ## performing cat_id filtering here.
    cat_id =  test_dataset.disease_to_idx(disease) if disease else None

    cat_ids = (
        [
            test_dataset.disease_to_idx(d)
            for d in test_dataset.labels_cols
        ]
        if cat_id is None
        else [cat_id]
    )

    if not (cat_ids is None):
        normal_eval_params_dict["bbox"].catIds = cat_ids
        normal_eval_params_dict["segm"].catIds = cat_ids
        
    test_evaluator, _ = xami_evaluate(
                model,
                test_dataloader,
                device=device,
                params_dict=normal_eval_params_dict,
                coco=test_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

    num_fps[naming_map[select_model]]= get_num_fps(
                test_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )

    num_fns[naming_map[select_model]]= get_num_fns(
                test_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )

    num_tps[naming_map[select_model]]= get_num_tps(
                test_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )


  0%|          | 0/4 [00:00<?, ?it/s]

Load custom model
Using pretrained backbone. mobilenet_v3
Using pretrained backbone. mobilenet_v3
Mask Hidden Layers 256
Using SGD as optimizer with lr=0.001
creating index...
index created!
creating index...
index created!
Evaluation:  [ 0/26]  eta: 0:01:26  loss: 0.7544 (0.7544)  loss_classifier: 0.0243 (0.0243)  loss_box_reg: 0.0139 (0.0139)  loss_mask: 0.6970 (0.6970)  loss_objectness: 0.0180 (0.0180)  loss_rpn_box_reg: 0.0012 (0.0012)  model_time: 2.2337 (2.2337)  evaluator_time: 0.2845 (0.2845)  time: 3.3245  data: 0.5346  max mem: 1060


 25%|██▌       | 1/4 [00:53<02:41, 53.93s/it]

Evaluation:  [25/26]  eta: 0:00:00  loss: 0.4686 (0.4277)  loss_classifier: 0.0155 (0.0194)  loss_box_reg: 0.0091 (0.0133)  loss_mask: 0.3755 (0.3700)  loss_objectness: 0.0176 (0.0222)  loss_rpn_box_reg: 0.0015 (0.0029)  model_time: 0.1157 (0.2011)  evaluator_time: 0.1753 (0.1844)  time: 0.8857  data: 0.4988  max mem: 1178
Evaluation: Total time: 0:00:25 (0.9744 s / it)
Averaged stats: loss: 0.4686 (0.4277)  loss_classifier: 0.0155 (0.0194)  loss_box_reg: 0.0091 (0.0133)  loss_mask: 0.3755 (0.3700)  loss_objectness: 0.0176 (0.0222)  loss_rpn_box_reg: 0.0015 (0.0029)  model_time: 0.1157 (0.2011)  evaluator_time: 0.1753 (0.1844)
Accumulating evaluation results...
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.00s).
IoU metric: bbox
 Average Precision  (AP) @[ IoBB=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoBB=0.50      | area=   all | maxDets= 10 ] = 0.193
 Average Precision  (AP) @[ IoBB=0.75      | area=   all | maxDets= 10 ] = 0.168
 A

 50%|█████     | 2/4 [01:44<01:43, 51.73s/it]

Evaluation:  [25/26]  eta: 0:00:00  loss: 0.4028 (0.3135)  loss_classifier: 0.0092 (0.0129)  loss_box_reg: 0.0064 (0.0093)  loss_mask: 0.3421 (0.2649)  loss_objectness: 0.0160 (0.0233)  loss_rpn_box_reg: 0.0014 (0.0031)  model_time: 0.0867 (0.0829)  evaluator_time: 0.1450 (0.1332)  time: 0.8247  data: 0.5369  max mem: 1178
Evaluation: Total time: 0:00:21 (0.8307 s / it)
Averaged stats: loss: 0.4028 (0.3135)  loss_classifier: 0.0092 (0.0129)  loss_box_reg: 0.0064 (0.0093)  loss_mask: 0.3421 (0.2649)  loss_objectness: 0.0160 (0.0233)  loss_rpn_box_reg: 0.0014 (0.0031)  model_time: 0.0867 (0.0829)  evaluator_time: 0.1450 (0.1332)
Accumulating evaluation results...
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.00s).
IoU metric: bbox
 Average Precision  (AP) @[ IoBB=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoBB=0.50      | area=   all | maxDets= 10 ] = 0.098
 Average Precision  (AP) @[ IoBB=0.75      | area=   all | maxDets= 10 ] = 0.035
 A

 75%|███████▌  | 3/4 [02:27<00:47, 47.70s/it]

Evaluation:  [25/26]  eta: 0:00:00  loss: 0.0516 (0.0571)  loss_classifier: 0.0164 (0.0197)  loss_box_reg: 0.0113 (0.0139)  loss_objectness: 0.0154 (0.0206)  loss_rpn_box_reg: 0.0017 (0.0029)  model_time: 0.0692 (0.0691)  evaluator_time: 0.0010 (0.0008)  time: 0.6356  data: 0.5084  max mem: 1178
Evaluation: Total time: 0:00:16 (0.6473 s / it)
Averaged stats: loss: 0.0516 (0.0571)  loss_classifier: 0.0164 (0.0197)  loss_box_reg: 0.0113 (0.0139)  loss_objectness: 0.0154 (0.0206)  loss_rpn_box_reg: 0.0017 (0.0029)  model_time: 0.0692 (0.0691)  evaluator_time: 0.0010 (0.0008)
Accumulating evaluation results...
DONE (t=0.00s).
IoU metric: bbox
 Average Precision  (AP) @[ IoBB=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoBB=0.50      | area=   all | maxDets= 10 ] = 0.323
 Average Precision  (AP) @[ IoBB=0.75      | area=   all | maxDets= 10 ] = 0.138
 Average Precision  (AP) @[ IoBB=0.50:0.95 | area= small | maxDets= 10 ] = -1.000
 Average Precision  (AP) @[

100%|██████████| 4/4 [03:20<00:00, 50.04s/it]

Evaluation:  [25/26]  eta: 0:00:00  loss: 0.4317 (0.3738)  loss_classifier: 0.0255 (0.0283)  loss_box_reg: 0.0093 (0.0165)  loss_mask: 0.3599 (0.3044)  loss_objectness: 0.0200 (0.0219)  loss_rpn_box_reg: 0.0014 (0.0028)  model_time: 0.1067 (0.1147)  evaluator_time: 0.2289 (0.2512)  time: 0.9625  data: 0.5241  max mem: 1368
Evaluation: Total time: 0:00:25 (0.9839 s / it)
Averaged stats: loss: 0.4317 (0.3738)  loss_classifier: 0.0255 (0.0283)  loss_box_reg: 0.0093 (0.0165)  loss_mask: 0.3599 (0.3044)  loss_objectness: 0.0200 (0.0219)  loss_rpn_box_reg: 0.0014 (0.0028)  model_time: 0.1067 (0.1147)  evaluator_time: 0.2289 (0.2512)
Accumulating evaluation results...
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
IoU metric: bbox
 Average Precision  (AP) @[ IoBB=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoBB=0.50      | area=   all | maxDets= 10 ] = 0.086
 Average Precision  (AP) @[ IoBB=0.75      | area=   all | maxDets= 10 ] = 0.049
 A

In [10]:
print(disease if disease else "Overall")
pd.DataFrame({
    "#False Positives": num_fps,
    "#False Negatives": num_fns,
    "#True Positives": num_tps,
}).astype(int)

Pulmonary edema


,#False Positives,#False Negatives,#True Positives
CXR+Clinical (fusion1),51,8,10
CXR+Clinical (fusion2),23,13,5
CXR+Clinical (fusion1+2),45,7,11
CXR (baseline),75,7,11
